<a href="https://colab.research.google.com/github/TheOneTrueGuy/DigitalDayDream/blob/main/ClipLoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Digital daydreaming in latent space with CLIP Interogator and Stable Diffusion

In [ ]:
#@title Check GPU
!nvidia-smi -L

In [ ]:
#@title Installation
!sudo apt update
import os, subprocess, sys

def setup():
    install_cmds = [
        ['pip', 'install', 'diffusers==0.10.0'],
        ['pip', 'install', 'gradio', ],
        ['pip', 'install', '-e', 'git+https://github.com/MirageML/BLIP.git@main#egg=blip'],#  https://github.com/pharmapsychotic/BLIP.git@lib#egg=blip'],
        ['pip', 'install', 'clip-interrogator==0.3.1' ]
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()



# save time of preprocessing by downloading cache from huggingface
print("Download preprocessed cache files...")
CACHE_URLS = [
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_artists.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_flavors.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_mediums.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_movements.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_trendings.pkl',
]
os.makedirs('cache', exist_ok=True)
for url in CACHE_URLS:
    print(subprocess.run(['wget', url, '-P', 'cache'], stdout=subprocess.PIPE).stdout.decode('utf-8'))

sys.path.append('src/blip')

!pip install --upgrade diffusers transformers accelerate

from clip_interrogator import Config, Interrogator

ci = Interrogator(Config())



In [ ]:
#@title Mount Google Drive
import os
from google.colab import drive

drive.mount('/content/drive')
results_dir = '/content/drive/MyDrive/clip-loop'
os.makedirs(results_dir, exist_ok=True)

# fetch image_to_image.py
!git clone https://github.com/TheOneTrueGuy/DigitalDayDream.git
!cp DigitalDayDream/image_to_image.py /content/
!cp DigitalDayDream/noiser.py /content/
!cp DigitalDayDream/replace.txt /content/

print(f"\nResults will be saved to {results_dir}")
!pip install accelerate

In [ ]:
#@title Login to huggingface
!huggingface-cli login

In [ ]:
#@title Create animation!

start_image_path = "/content/st-0005.png" #@param {type:"string"}
style_prompt = "stylish style from Styleland" #@param {type:"string"}
dir_name = "output_dir" #@param {type:"string"}
max_frames = 200 #@param {type:"integer"}

import datetime
import os
import torch
import numpy as np

from PIL import Image

from diffusers import StableDiffusionImg2ImgPipeline

class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)


rond=np.random.randint(7,100000000)
generator = torch.Generator("cuda").manual_seed(rond)
print( "random seed " + str(rond))

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.safety_checker = FakeSafety() #remove/comment out this line to restore adult content filtering

os.makedirs(dir_name, exist_ok=True)
dialog = open("dialog.txt", "a", encoding="utf-8")
dialog.write("random seed:"+str(rond) + "\n")
for xn in range(0, max_frames, 1):
  im = Image.open(start_image_path if xn == 0 else "prev.png").convert("RGB")
  im = im.resize((512,512))

  cliprompt = ci.interrogate_fast(im)
  if style_prompt:
    cliprompt = cliprompt.split(", ")
    cliprompt.insert(1, style_prompt)
    cliprompt = ", ".join(cliprompt)
  print(f"{xn:04d}: {cliprompt}\n")  
  dialog.write(f"{xn:04d}: {cliprompt},\n")
  
  image = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]
  image.save("prev.png")
  os.system("python noiser.py -p prev.png -n " +str(int(xn*2.8)))
  image.save(f"{dir_name}/{xn:04d}.png")
#  prog_noise("prev.png", xn, 2.8)
  #os.system("bash 1julien.sh st.png zi") #my homemade zoom function
  # and you will have to compile the video with ffmpeg or something
dialog.close()

# zip animation frames and put in Google Drive
year_month_day = datetime.datetime.now().strftime("%Y_%m_%d")
zip_name = f"{year_month_day}_{dir_name}.zip"

!mv dialog.txt $dir_name/
!zip -r $zip_name $dir_name/ 
!cp $zip_name $results_dir

print(f"Animation frames exported to {results_dir}/{zip_name}")


 painting : photorealistic image rendered giza : Uxmal  desert, : jungle,  painting : rendered image  drawing : black and white photo : colorful stucco  egyptian men : elegant parrots on, : on surrounded by techno jungle,  dreadlocks : feathers skull:skully skull:white turnip apples:crystals green:green and red fractal:neon liquid paint splash psychedelic:asymmetric fractal kaleidoscop spiral:maze puzzle
 woman:flower headphones on:mechanical tendrils a close up : wide angle a close up of a person wearing headphones:wide angle view of a factory

In [ ]:
#run this cell after starting cell above if you don't want to burn 
#google compute credits unnecessarily.
from google.colab import runtime
runtime.unassign()

In [ ]:
year_month_day = datetime.datetime.now().strftime("%Y_%m_%d")
zip_name = f"{year_month_day}_{dir_name}.zip"
!mv dialog.txt $dir_name/
!zip -r $zip_name $dir_name/ 
!cp $zip_name $results_dir


In [ ]:
#working scrap section, sorry 
file2 = open("moto1.txt")
lines2 = file2.read().replace("\n", " ")
file2.close()
moto=lines2.split(" ")
lenmo=len(moto)